# 📚 Fine-Tuning de um Modelo BERT Adaptado para Reconhecimento de Entidades Nomeadas (NER)

Este notebook o treinamento de um modelo **BERT pré-ajustado com Masked Language Modeling (MLM)** em textos legislativos para treinar uma tarefa de **Reconhecimento de Entidades Nomeadas (NER)**, usando a biblioteca **Hugging Face Transformers**.

A abordagem parte de um modelo BERT em português (pré-treinado com MLM no corpus **UlyssesNER-Br**) para explorar todo o **vocabulário jurídico e legislativo**, garantindo melhor adaptação ao domínio.  
A partir desse ponto, o notebook aplica um ajuste fino (fine-tuning) para treinar o modelo a **detectar e rotular entidades** como **PESSOA**, **ORGANIZAÇÃO**, **DATA**, entre outras, em textos reais da Câmara dos Deputados.

Ao longo do notebook, realizamos:
- **Montagem do Google Drive:** Importação do modelo BERT ajustado com MLM.
- **Carregamento e preparação de dados NER:** Download do corpus anotado, mapeamento de rótulos BIO e tokenização alinhada.
- **Configuração do treino:** Definição do `Trainer` para token classification com métricas de sequência (SeqEval).
- **Treinamento supervisionado:** Ajuste fino do BERT para NER.
- **Salvar o modelo final:** Armazenamento do modelo ajustado no Google Drive para uso posterior.

O resultado é um **BERT especializado** capaz de **reconhecer automaticamente entidades jurídicas em português**, servindo como base para sistemas de extração de informação em textos legislativos.

Este notebook serve para comparar a abordagem baseada em Transformers com outra baseada na biblioteca spaCy para a tarefa de NER.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp /content/drive/MyDrive/bert-finetuned-ulysses.zip /content/

In [ ]:
import zipfile
from pathlib import Path

zip_path = Path("/content/bert-finetuned-ulysses.zip")
extract_dir = Path("/content/bert-finetuned-ulysses")

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"✅ Extraído em: {extract_dir}")


✅ Extraído em: /content/bert-finetuned-ulysses


In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    AutoModelForMaskedLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)
from datasets import Dataset
from pathlib import Path
import requests
import json
import torch
import gc

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
path_to_save_ner = Path("./outputs/bert_ulysses_ner")
path_to_save_ner.mkdir(parents=True, exist_ok=True)

In [ ]:
base_url = "https://raw.githubusercontent.com/Convenio-Camara-dos-Deputados/ulyssesner-br-propor/main/PL-corpus_v2/ulysses_categories/holdout/"
urls = {
    "train": base_url + "train.json",
    "dev": base_url + "dev.json",
    "test": base_url + "test.json",
}

def load_data(url):
    r = requests.get(url)
    return json.loads(r.text)

train_data = load_data(urls["train"])
dev_data = load_data(urls["dev"])
test_data = load_data(urls["test"])


In [ ]:
def prepare_dataset(data):
    return {
        "tokens": [ex["tokens"] for ex in data],
        "ner_tags": [ex["ner_tokens"] for ex in data],
    }

train_dict = prepare_dataset(train_data)
dev_dict = prepare_dataset(dev_data)
test_dict = prepare_dataset(test_data)

train_dataset = Dataset.from_dict(train_dict)
dev_dataset = Dataset.from_dict(dev_dict)
test_dataset = Dataset.from_dict(test_dict)

In [ ]:
unique_labels = list(set(label for doc in train_dict["ner_tags"] for label in doc))
unique_labels.sort()
label2id = {label: i for i, label in enumerate(unique_labels)}
id2label = {i: label for label, i in label2id.items()}

print("id2label:", id2label)


id2label: {0: 'B-DATA', 1: 'B-EVENTO', 2: 'B-FUNDAMENTO', 3: 'B-LOCAL', 4: 'B-ORGANIZACAO', 5: 'B-PESSOA', 6: 'B-PRODUTODELEI', 7: 'I-DATA', 8: 'I-EVENTO', 9: 'I-FUNDAMENTO', 10: 'I-LOCAL', 11: 'I-ORGANIZACAO', 12: 'I-PESSOA', 13: 'I-PRODUTODELEI', 14: 'O'}


In [ ]:
model_checkpoint = Path(
    "/content/bert-finetuned-ulysses"
)

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at /content/bert-finetuned-ulysses and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
label_all_tokens = True

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True, max_length=128
    )
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label2id[label[word_idx]])
            else:
                label_ids.append(label2id[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_train = train_dataset.map(tokenize_and_align_labels, batched=True)
tokenized_dev = dev_dataset.map(tokenize_and_align_labels, batched=True)
tokenized_test = test_dataset.map(tokenize_and_align_labels, batched=True)


Map:   0%|          | 0/1760 [00:00<?, ? examples/s]

Map:   0%|          | 0/140 [00:00<?, ? examples/s]

Map:   0%|          | 0/592 [00:00<?, ? examples/s]

In [ ]:
!pip install evaluate

In [ ]:
!pip install seqeval

In [ ]:
import evaluate
import numpy as np

metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
        "eval_f1": results["overall_f1"],
    }



In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

model_name = str(model_checkpoint).split("/")[-1]

training_args = TrainingArguments(
    output_dir=path_to_save_ner / f"{model_name}-ner",
    overwrite_output_dir=True,
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=1,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=1,
    logging_steps=5,
    logging_strategy="steps",
    eval_strategy="epoch",
    save_strategy="epoch",
    eval_steps=None,
    save_steps=None,
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    seed=271828,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_dev,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

/tmp/ipython-input-30-1051715750.py:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,F1,Precision,Recall,Accuracy
1,0.169100,0.097358,0.822034,0.822034,0.822034,0.976423
2,0.001600,0.122304,0.807692,0.814655,0.800847,0.973376
3,0.000400,0.111022,0.830898,0.818930,0.843220,0.978348
4,0.000500,0.127228,0.863732,0.854772,0.872881,0.979310
5,0.000200,0.130753,0.866808,0.864979,0.868644,0.980273


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-sco

TrainOutput(global_step=8800, training_loss=0.061117967507207965, metrics={'train_runtime': 18145.125, 'train_samples_per_second': 0.485, 'train_steps_per_second': 0.485, 'total_flos': 223879037262300.0, 'train_loss': 0.061117967507207965, 'epoch': 5.0})

In [ ]:
output_dir = "/content/drive/MyDrive/meu_modelo_ner"
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

('/content/drive/MyDrive/meu_modelo_ner/tokenizer_config.json',
 '/content/drive/MyDrive/meu_modelo_ner/special_tokens_map.json',
 '/content/drive/MyDrive/meu_modelo_ner/vocab.txt',
 '/content/drive/MyDrive/meu_modelo_ner/added_tokens.json',
 '/content/drive/MyDrive/meu_modelo_ner/tokenizer.json')

In [ ]:
print(path_to_save_ner / f"{model_name}-lener")

outputs/bert_ulysses_ner/bert-finetuned-ulysses-lener
